# EDINETのAPIを使用してみる
本書のゴール＝四半期報告書のpdfを取得する

以下のコンテンツではEDINET閲覧サイト（[url](https://disclosure2.edinet-fsa.go.jp/WEEE0030.aspx?bXVsPeWVhuiIueS4ieS6lSZjdGY9b2ZmJmZscz1vbiZscHI9b2ZmJnJwcj1vZmYmb3RoPW9mZiZwZnM9NiZ5ZXI9Jm1vbj0=)）から取得できる情報をもとにデータを加工して作成しています。本ページに記載されているコードの実行や取得したデータの利用についてはEDINETの利用規約を確認し理解した上で利用してください。また、Version2のAPIを使用する際はログイン認証＋APIキーの発行が必要になります。

※注意：使用時には必ず利用規約（[pdf](https://disclosure2dl.edinet-fsa.go.jp/guide/static/disclosure/download/ESE140191.pdf)・[web](https://disclosure2dl.edinet-fsa.go.jp/guide/static/submit/WZEK0030.html))を確認した上で常識の範囲内での使用にとどめてください。

In [1]:
import os
import requests
import pandas as pd

API_ENDPOINT = "https://disclosure.edinet-fsa.go.jp/api/v2"  # v2を使用する

### 書類一覧APIのリクエストURL
11_EDINET_API仕様書に従って書類一覧を取得してみます。

In [2]:
request_params = {
    "date": "2024-02-09",
    "type": 2,  # 1=メタデータのみ、2=提出書類一覧及びメタデータ
    "Subscription-Key": os.environ.get(
        "EDINET_API_KEY"
    ),  # v1を使用する場合は不要, 2024年３月29日（金）まで利用可能
}

docs_submitted_json = requests.get(
    f"{API_ENDPOINT}/documents.json", request_params
).json()

## 取得データの確認
今は四半期報告書を取得したいので、『四半期報告書』が文書の説明（`docDescription`）に含まれている行のみを抽出して確認します。

In [3]:
sd_df = pd.DataFrame(docs_submitted_json["results"])
sd_df = sd_df[sd_df["docDescription"].str.contains("四半期報告書", na=False)]
sd_df.head()

,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
1,2,S100SSMQ,E04505,95070,9470001001933,四国電力株式会社,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1
2,3,S100SSHR,E01441,59730,5122001016280,株式会社トーアミ,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1
5,6,S100SQAH,E30982,71750,9220001001223,今村証券株式会社,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1
6,7,S100SPH6,E03562,83600,3090001002315,株式会社 山梨中央銀行,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1
10,11,S100SRF2,E00783,40910,7010701015826,日本酸素ホールディングス株式会社,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1


### DocIDの確認
商船三井の四半期報告書のDocIDを確認してみます。`filerName`に商船三井が含まれる行のみを抽出します。

In [4]:
sd_df[sd_df.filerName.str.contains("商船三井")]

,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
1307,1308,S100STH6,E04236,91040,4010401082896,株式会社商船三井,None,010,043000,140,...,None,0,0,0,1,1,0,0,1,1


### 四半期報告書の取得

`docID` をもとに四半期報告書の取得に必要なデータを取得します。pdfを取得したい場合はマニュアルに従い、type=2を指定します。

In [5]:
docID = "S100STH6"
pdf_response = requests.get(
    f"{API_ENDPOINT}/documents/{docID}",
    {
        "type": 2,
        "Subscription-Key": os.environ.get("EDINET_API_KEY"),
    },
)

with open("sample.pdf", "wb") as f:
    f.write(pdf_response.content)

#### ダウンロードしたpdfを表示する

In [6]:
from IPython.display import display_pdf

with open("sample.pdf", "rb") as f:
    display_pdf(f.read(), raw=True)